In [1]:
import time
import numpy as np

import _init_paths
from libs.scheduler import PfScheduler
from libs.models.naive_solver import NaiveURLLCSolver
from libs.models.greedy_solver import GreedyURLLCSolver
from utils.generator import generate
from utils.metrics import get_embb_utility
# from utils.visualization import draw_matrix

D:\Development\Code\URLLC\Joint-eMBB-URLLC
D:\Development\Code\URLLC\Joint-eMBB-URLLC\libs\models


In [2]:
# parameter setting
rb_size = 15
rb_num = 254
embb_num = 3
embb_slot_len = 7
urllc_num = 6
urllc_slot_len = 1
max_sim_time_slot_len = embb_slot_len

embb_rb_req = [100, 50, 120]
embb_rb_size = [6000, 12000, 3000]
urllc_rb_req = [10, 10, 10, 10, 10, 10]
urllc_rb_size = [30, 30, 30, 30, 30, 30]
urllc_slot_start = [1, 2, 3, 4, 5, 6]

In [9]:
#urllc scheduler setting
URLLCSolver = NaiveURLLCSolver
# URLLCSolver = GreedyURLLCSolver

# generate
embb_users, urllc_users, RB_map = generate(rb_size,
    rb_num,
    embb_num, 
    embb_slot_len, 
    urllc_num,
    urllc_slot_len,
    embb_rb_req,
    embb_rb_size,
    urllc_rb_req,
    urllc_rb_size,
    urllc_slot_start,
    latency=1,
    error_rate=1e-5,
    mcs_error=1e-5,
)


In [10]:
embb_users[0].__dict__.items()

dict_items([('user_info', {'id': 1}), ('rb_size', 6000), ('rb_num_req', 100), ('rb_num_ass', 0), ('rb_start', 0), ('sche_times', 0), ('active', 1), ('slot_len', 7), ('DRC', 85.71428571428571), ('rate_slot', 0.0), ('rate_avg', 0.0), ('rate_cur', 0.0), ('replace_num', 0)])

In [11]:
global_time = 0
global_timeout_urllc_users = []

# embb_time_slot pfs scheduler
pf_scheduler = PfScheduler(RB_map, embb_users)
pf_scheduler.allocate_resource()


    
# urllc_time_slot get urllc_active_come_list, setting urllc come from global_time=1 and later
urllc_users.sort(key=lambda x:x.slot_start)
urllc_come_time = np.array([u.slot_start for u in urllc_users])

global_time = 0
global_timeout_urllc_users = []


In [18]:
urllc_come_time

array([1, 2, 3, 4, 5, 6])

In [13]:
global_time

0

In [14]:
embb_users[0].__dict__.items()

dict_items([('user_info', {'id': 1}), ('rb_size', 6000), ('rb_num_req', 100), ('rb_num_ass', 100), ('rb_start', 0), ('sche_times', 1), ('active', 1), ('slot_len', 7), ('DRC', 85.71428571428571), ('rate_slot', 0.0), ('rate_avg', 85.71428571428571), ('rate_cur', 85.71428571428571), ('replace_num', 0)])

In [15]:
# urllc_time_slot get urllc_active_come_list, setting urllc come from global_time=1 and later
urllc_users.sort(key=lambda x:x.slot_start)
urllc_come_time = np.array([u.slot_start for u in urllc_users])

delay_users = []
timer = []
# urllc scheduler loop for all the urllc_time_slot without considering embb user rescheduling
while(global_time < max_sim_time_slot_len):
    global_time += 1
    indexes = np.where(urllc_come_time==global_time)[0]
    if len(indexes) == 0:
        continue
    urllc_users_list = delay_users + urllc_users.copy()[indexes[0]:indexes[-1]+1]

    
    urllc_scheduler = URLLCSolver(RB_map, embb_users, urllc_users_list)
    
    start = time.time()
    ass_users, delay_users, timeout_users = urllc_scheduler.allocate_resource()
    timer.append(time.time() - start)

    global_timeout_urllc_users.append(timeout_users)
    
    print(RB_map.__dict__.items())
    # urllc leave
    urllc_scheduler.leave(ass_users)
    
# get_embb_utility miss_list and time cost for all the urllc scheduler
embb_utility = get_embb_utility(embb_users)
total_time_cost = sum(timer)

# visualize (TODO)
for user in embb_users:
    print(user .__dict__.items())
print("\n")
print(RB_map.__dict__.items())

# next embb_time_slot (not show here)


dict_items([('rb_avi', 254), ('rb_size', 30), ('bitmap', array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int64)), ('border_bitmap_id', 3)])
dict_items([('rb_avi', 254), ('rb_size', 30), ('bitmap'

In [21]:
global_time

7

In [8]:
embb_users[0].__dict__.items()

dict_items([('user_info', {'id': 1}), ('rb_size', 6000), ('rb_num_req', 100), ('rb_num_ass', 100), ('rb_start', 0), ('sche_times', 1), ('active', 1), ('slot_len', 7), ('DRC', 85.71428571428571), ('rate_slot', 0.0), ('rate_avg', 78.62479293564512), ('rate_cur', 78.62479293564512), ('replace_num', 0)])

In [9]:
# embb_time_slot pfs scheduler
pf_scheduler = PfScheduler(RB_map, embb_users)
pf_scheduler.allocate_resource()


In [10]:
for user in embb_users:
    print(user .__dict__.items())

dict_items([('user_info', {'id': 1}), ('rb_size', 6000), ('rb_num_req', 100), ('rb_num_ass', 100), ('rb_start', 120), ('sche_times', 2), ('active', 1), ('slot_len', 7), ('DRC', 85.71428571428571), ('rate_slot', 0.0), ('rate_avg', 82.16953932496541), ('rate_cur', 85.71428571428571), ('replace_num', 0)])
dict_items([('user_info', {'id': 2}), ('rb_size', 12000), ('rb_num_req', 50), ('rb_num_ass', 34), ('rb_start', 220), ('sche_times', 2), ('active', 1), ('slot_len', 7), ('DRC', 85.71428571428571), ('rate_slot', 0.0), ('rate_avg', 72.0), ('rate_cur', 58.285714285714285), ('replace_num', 0)])
dict_items([('user_info', {'id': 3}), ('rb_size', 3000), ('rb_num_req', 120), ('rb_num_ass', 120), ('rb_start', 0), ('sche_times', 2), ('active', 1), ('slot_len', 7), ('DRC', 51.42857142857143), ('rate_slot', 0.0), ('rate_avg', 48.0), ('rate_cur', 51.42857142857143), ('replace_num', 0)])


In [11]:
embb_utility

950.9110637881344

### eMBB only without URLLC

In [5]:
for i in range(240000):
    pf_scheduler = PfScheduler(RB_map, embb_users)
    pf_scheduler.allocate_resource()

In [6]:
for user in embb_users:
    print(user .__dict__.items())

dict_items([('user_info', {'id': 1}), ('rb_size', 6000), ('rb_num_req', 100), ('rb_num_ass', 84), ('rb_start', 170), ('sche_times', 240000), ('active', 1), ('slot_len', 7), ('DRC', 85.71428571428571), ('rate_slot', 0.0), ('rate_avg', 80.63491428567671), ('rate_cur', 72.0), ('replace_num', 0)])
dict_items([('user_info', {'id': 2}), ('rb_size', 12000), ('rb_num_req', 50), ('rb_num_ass', 50), ('rb_start', 0), ('sche_times', 240000), ('active', 1), ('slot_len', 7), ('DRC', 85.71428571428571), ('rate_slot', 0.0), ('rate_avg', 80.63485714282145), ('rate_cur', 85.71428571428571), ('replace_num', 0)])
dict_items([('user_info', {'id': 3}), ('rb_size', 3000), ('rb_num_req', 120), ('rb_num_ass', 120), ('rb_start', 50), ('sche_times', 240000), ('active', 1), ('slot_len', 7), ('DRC', 51.42857142857143), ('rate_slot', 0.0), ('rate_avg', 48.38097142857368), ('rate_cur', 51.42857142857143), ('replace_num', 0)])


In [7]:
embb_utility = get_embb_utility(embb_users)

In [8]:
embb_utility

952.0565753732992

In [33]:
for i in range(len(urllc_slot_start)):
    urllc_slot_start[i] += global_time

In [34]:
urllc_slot_start

[8, 9, 10, 11, 12, 13]

### Jointly eMBB and URLLC traffic without retransmission 

In [ ]:
# parameter setting
rb_size = 15
rb_num = 254
embb_num = 3
embb_slot_len = 7
urllc_num = 6
urllc_slot_len = 1
id_total = 0
max_sim_time_slot_len = embb_slot_len

embb_rb_req = [100, 50, 120]
embb_rb_size = [6000, 12000, 3000]
urllc_rb_req = [10, 10, 10, 10, 10, 10]
urllc_rb_size = [30, 30, 30, 30, 30, 30]
urllc_slot_start = [1, 2, 3, 4, 5, 6]

#urllc scheduler setting
URLLCSolver = NaiveURLLCSolver
# URLLCSolver = GreedyURLLCSolver

global_time = 0
global_timeout_urllc_users = []

In [ ]:
# generate eMBB
embb_users, urllc_users, RB_map, id_total = generate(rb_size,
    rb_num,
    embb_num, 
    embb_slot_len, 
    urllc_num,
    urllc_slot_len,
    embb_rb_req,
    embb_rb_size,
    urllc_rb_req,
    urllc_rb_size,
    urllc_slot_start,
    latency=1,
    error_rate=1e-5,
    mcs_error=1e-5,
)


In [ ]:
# embb_time_slot pfs scheduler
pf_scheduler = PfScheduler(RB_map, embb_users)
pf_scheduler.allocate_resource()


    
# urllc_time_slot get urllc_active_come_list, setting urllc come from global_time=1 and later
urllc_users.sort(key=lambda x:x.slot_start)
urllc_come_time = np.array([u.slot_start for u in urllc_users])

In [ ]:
delay_users = []
timer = []
# urllc scheduler loop for all the urllc_time_slot without considering embb user rescheduling
while(global_time < max_sim_time_slot_len):
    global_time += 1
    indexes = np.where(urllc_come_time==global_time)[0]
    if len(indexes) == 0:
        continue
    urllc_users_list = delay_users + urllc_users.copy()[indexes[0]:indexes[-1]+1]

    
    urllc_scheduler = URLLCSolver(RB_map, embb_users, urllc_users_list)
    
    start = time.time()
    ass_users, delay_users, timeout_users = urllc_scheduler.allocate_resource()
    timer.append(time.time() - start)

    global_timeout_urllc_users.append(timeout_users)
    
    print(RB_map.__dict__.items())
    # urllc leave
    urllc_scheduler.leave(ass_users)
    
# get_embb_utility miss_list and time cost for all the urllc scheduler
embb_utility = get_embb_utility(embb_users)
total_time_cost = sum(timer)

# visualize (TODO)
for user in embb_users:
    print(user .__dict__.items())
print("\n")
print(RB_map.__dict__.items())

# next embb_time_slot (not show here)

In [ ]:
#update URLLC
for i in range(len(urllc_slot_start)):
    urllc_slot_start[i] += global_time

urllc_users, id_total = generate(
    urllc_num,
    urllc_slot_len,
    urllc_rb_req,
    urllc_rb_size,
    urllc_slot_start,
    latency=1,
    error_rate=1e-5,
    mcs_error=1e-5,
    id_total
)

